In [1]:
from keras.layers import Input, Lambda, Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
IMAGE_SIZE = [224,224]

train_path = 'chest_xray/train'
valid_path  = 'chest_xray/test'

In [3]:
vgg= VGG16(input_shape = IMAGE_SIZE +[3], weights = 'imagenet',include_top = False)

58892288/58889256 [==============================] - 20s 0us/step


In [5]:
for layer in vgg.layers:
    layer.trainable = False


In [6]:
folders = glob("chest_xray/train/*")

In [7]:
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders),activation='softmax')(x)
model = Model(inputs=vgg.input,outputs=prediction)

In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [11]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./225, shear_range = 0.3,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
training_set = train_datagen.flow_from_directory('chest_xray/train',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 5216 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory('chest_xray/test',target_size=(224,224),batch_size=32,class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(training_set,validation_data=test_set,epochs =5,steps_per_epoch=len(training_set),validation_steps=len(test_set))

Epoch 1/5
163/163 [==============================] - 540s 3s/step - loss: 0.1809 - accuracy: 0.9294 - val_loss: 0.2274 - val_accuracy: 0.9167
Epoch 2/5
163/163 [==============================] - 569s 3s/step - loss: 0.1106 - accuracy: 0.9567 - val_loss: 0.4176 - val_accuracy: 0.9135
Epoch 3/5
163/163 [==============================] - 562s 3s/step - loss: 0.0942 - accuracy: 0.9638 - val_loss: 0.2968 - val_accuracy: 0.9151
Epoch 4/5
163/163 [==============================] - 563s 3s/step - loss: 0.0989 - accuracy: 0.9636 - val_loss: 0.1124 - val_accuracy: 0.9231
Epoch 5/5
162/163 [============================>.] - ETA: 3s - loss: 0.0752 - accuracy: 0.9726

In [ ]:
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='val loss')
plt.legent()
plt.show()
plt.savefig('LossVal_loss')

plt.plot(r.history['acc'],label='train acc')
plt.plot(r.history['val_acc'],label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('vgg16.h5')